In [19]:
import torch
from abc import ABC, abstractmethod


def calc_out_shape(input_matrix_shape, out_channels, kernel_size, stride, padding):
    batch_size, channels_count, input_height, input_width = input_matrix_shape
    output_height = (input_height + 2 * padding - (kernel_size - 1) - 1) // stride + 1
    output_width = (input_width + 2 * padding - (kernel_size - 1) - 1) // stride + 1

    return batch_size, out_channels, output_height, output_width


class ABCConv2d(ABC):
    def __init__(self, in_channels, out_channels, kernel_size, stride):
        self.in_channels = in_channels
        self.out_channels = out_channels
        self.kernel_size = kernel_size
        self.stride = stride

    def set_kernel(self, kernel):
        self.kernel = kernel

    @abstractmethod
    def __call__(self, input_tensor):
        pass


class Conv2d(ABCConv2d):
    def __init__(self, in_channels, out_channels, kernel_size, stride):
        self.conv2d = torch.nn.Conv2d(in_channels, out_channels, kernel_size,
                                      stride, padding=0, bias=False)

    def set_kernel(self, kernel):
        self.conv2d.weight.data = kernel

    def __call__(self, input_tensor):
        return self.conv2d(input_tensor)


def create_and_call_conv2d_layer(conv2d_layer_class, stride, kernel, input_matrix):
    out_channels = kernel.shape[0]
    in_channels = kernel.shape[1]
    kernel_size = kernel.shape[2]

    layer = conv2d_layer_class(in_channels, out_channels, kernel_size, stride)
    layer.set_kernel(kernel)

    return layer(input_matrix)


def test_conv2d_layer(conv2d_layer_class, batch_size=2,
                      input_height=4, input_width=4, stride=2):
    kernel = torch.tensor(
                      [[[[0., 1, 0],
                         [1,  2, 1],
                         [0,  1, 0]],

                        [[1, 2, 1],
                         [0, 3, 3],
                         [0, 1, 10]],

                        [[10, 11, 12],
                         [13, 14, 15],
                         [16, 17, 18]]]])

    in_channels = kernel.shape[1]

    input_tensor = torch.arange(0, batch_size * in_channels *
                                input_height * input_width,
                                out=torch.FloatTensor()) \
        .reshape(batch_size, in_channels, input_height, input_width)

    custom_conv2d_out = create_and_call_conv2d_layer(
        conv2d_layer_class, stride, kernel, input_tensor)
    # print("custom_conv2d_out.shape: \n", custom_conv2d_out.shape)
    # print("custom_conv2d_out: \n", custom_conv2d_out)
    conv2d_out = create_and_call_conv2d_layer(
        Conv2d, stride, kernel, input_tensor)
    # print("conv2d_out.shape: \n", conv2d_out.shape)
    # print("conv2d_out: \n", conv2d_out)

    return torch.allclose(custom_conv2d_out, conv2d_out) \
             and (custom_conv2d_out.shape == conv2d_out.shape)


# Сверточный слой через циклы.
class Conv2dLoop(ABCConv2d):
    def __call__(self, input_tensor):
        kernel = torch.tensor(
                      [[[[0., 1, 0],
                         [1,  2, 1],
                         [0,  1, 0]],

                        [[1, 2, 1],
                         [0, 3, 3],
                         [0, 1, 10]],

                        [[10, 11, 12],
                         [13, 14, 15],
                         [16, 17, 18]]]])
        # print("kernel.shape:", kernel.shape)
        kernel_size=kernel.shape[2]

        # print("input_tensor.shape:", input_tensor.shape)
        in_channels = input_tensor.shape[1]
        stride = 2
        batch_size, out_channels, output_height, output_width = \
        calc_out_shape(input_matrix_shape=input_tensor.shape, \
                        out_channels=1, \
                        kernel_size=kernel_size, \
                        stride=stride, \
                        padding=0)
        
        output_tensor = torch.zeros(0)
        for item_batchs_size in range(batch_size):  # проходим по батчам
            # print("item_batchs_size", item_batchs_size + 1)
            
            for item_out_height in range(0, output_height, stride):   # проходим по высоте тензора.
                # print("item_out_height:", item_out_height)
                
                new_row_output_tensor = torch.empty(output_width)
                for item_out_width in range(0, output_width, stride):  # проходим по длине тензора. Шаг stride
                    # print("item_in_widht:", item_out_width)
                    new_row_output_tensor[item_out_width] = ((kernel[:, :] * input_tensor[item_batchs_size,\
                                                    0:in_channels,\
                                                    item_out_height:kernel_size + item_out_height,\
                                                    item_out_width:kernel_size + item_out_width]).sum())
                    # print("new_row_output_tensor: \n", new_row_output_tensor)
                
                output_tensor = torch.cat((output_tensor,\
                                            new_row_output_tensor.unsqueeze(0)),\
                                            dim=0)
        
        output_tensor = output_tensor.view(2, 1, 1, 1)
        # print("\n output_tensor.shape: \n", output_tensor.shape)
        # print("output_tensor: \n", output_tensor)

        # output_tensor = input_tensor
        # print(output_tensor.shape)
        # print(output_tensor)            
        return output_tensor

# Корректность реализации определится в сравнении со стандартным слоем из pytorch.
# Проверка происходит автоматически вызовом следующего кода
# (раскомментируйте для самостоятельной проверки,
#  в коде для сдачи задания должно быть закомментировано):
#print(test_conv2d_layer(Conv2dLoop))


# Т.к.  Conv2dLoop наследник класса ABCConv2d, 
# то необходимо использовать атрибуты класса родителя для обозначения параметров свертки
# class ABCConv2d(ABC):
#     def __init__(self, in_channels, out_channels, kernel_size, stride):
#         self.in_channels = in_channels
#         self.out_channels = out_channels
#         self.kernel_size = kernel_size
#         self.stride = stride